In [13]:
import numpy as np
import pandas as pd
import cv2
import pathlib
from sklearn.manifold import TSNE

In [3]:
import plotly.graph_objects as go
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

In [4]:
# 良品
input_ok = r'C:\data\ok'
# 不良品
input_ng = r'C:\data\ng'
# 結果出力
result = r'C:\mapping'

In [5]:
# 画像サイズ
# 幅
width = 150
# 高さ
height = 150

In [12]:
# 画像ファイル名取得
ok_path = list(pathlib.Path(input_ok).glob('**/*.jpg'))
ng_path = list(pathlib.Path(input_ng).glob('**/*.jpg'))

In [53]:
# 画像読み込み
ok_array = [cv2.resize(cv2.imread(str(ok_path[i]), cv2.IMREAD_COLOR),
                       (width, height)).flatten()
            for i in range(len(ok_path))]
ng_array = [cv2.resize(cv2.imread(str(ng_path[i]), cv2.IMREAD_COLOR),
                       (width, height)).flatten()
           for i in range(len(ng_path))]

In [54]:
# 結合
all_array = np.concatenate([ok_array, ng_array], axis=0)
# ラベル付け
label = [0] * len(ok_path) + [1] * len(ng_path)

In [31]:
# ファイル名取得
ok_file = [ok_path[i].stem for i in range(len(ok_path))]
ng_file = [ng_path[i].stem for i in range(len(ng_path))]
file_name = ok_file + ng_file

In [61]:
X = pd.DataFrame(all_array)
y = pd.Series(label)

In [60]:
# t-SNE
X_reduced = TSNE(n_components=2, random_state=0).fit_transform(X)

In [68]:
df_reduced = pd.DataFrame(X_reduced, columns=['x', 'y'])
df = pd.concat([df_reduced, pd.Series(file_name, name='Name'),
                pd.Series(label, name='Label')], axis=1)

In [95]:
# 結果出力
df.to_csv(str(result) + r'\result.csv', index=False)

In [77]:
# テキスト設定
text0, text1 = [], []
for i in range(len(df)):
    if df.Label[i] == 0:
        text0.append('x: ' + str(round(df.x[i], 2)) + ', y: ' + str(round(df.y[i], 2))\
                    + '<br>ファイル名: ' + str(df.Name[i]))
    else:
        text1.append('x: ' + str(round(df.x[i], 2)) + ', y: ' + str(round(df.y[i], 2))\
                    + '<br>ファイル名: ' + str(df.Name[i]))

In [92]:
# plot
x0, y0 = X_reduced[y==0, 0], X_reduced[y==0, 1]
x1, y1 = X_reduced[y==1, 0], X_reduced[y==1, 1]
trace1 = go.Scatter(x=x0, y=y0, mode='markers', text=text0, hoverinfo='text', name='OK')
trace2 = go.Scatter(x=x1, y=y1, mode='markers', text=text1, hoverinfo='text', name='NG')
data = [trace1, trace2]
layout = go.Layout(hovermode='closest', title='t-SNE')
fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig, show_link=False)